In [1]:
import pandas as pd
import numpy as np
import re

# this methods reads filename and parses the data information
def readFileName(filename):
#     print(filename)
    dateObj = {}
    if('.html' in filename):
        result = re.search('(.*).html', filename)
        date = result.group(1)
        dateObj['year'] = date[0:4]
        dateObj['month']= date[5:7]
        dateObj['day'] = date[8:10]
        dateObj['page'] = date[len(date)-1]
    return dateObj
 

In [2]:
# this method is called for every file , it parses the data in the html file to stock and bond data frame
def fetchData(filename,folder,dateObj):
    print(folder)
    print(filename)
    
    import pandas as pd
    import numpy as np
    import codecs
    import re
    from bs4 import BeautifulSoup
    dateObj = readFileName(filename)
    with open('./'+folder+'/'+filename, 'rb') as f:
        html_doc = f.read()
    import lxml
    soup = BeautifulSoup(html_doc, "lxml")

    df_bond = pd.DataFrame()
    df_stock = pd.DataFrame()

    len_alltables = len(soup.find_all('table'))
    
    for table in soup.find_all('table'):
        row = -1
        colSpan = False
        rowspan = False
        nowProceed = False
        isBond = False;
        isStock = False;
        isNewBondRow = False
        isNewStockRow = False
#         city = ''
        
        if(len_alltables > 1):
            prev_sibling = table.find_previous_sibling('p')
            text = prev_sibling.text
            words = text.split()
            index = [i for i, s in enumerate(words) if 'STOCK' in s or 'SECURITIES' in s]
            if(len(index)):
                city = ' '.join(words[0:index[0]])

        df_bond_temp = pd.DataFrame()
        df_stock_temp = pd.DataFrame()
        for tr in table.find_all('tr'):
            countColumn = -1
            row = row+1
            countTD = len(tr.find_all('td'))
#             print(countTD)
            for td in tr.find_all('td'):
                countColumn = countColumn + 1

                if(countColumn == 0):
#                     print(td.get_text())
                    if(not(len(td.get_text()) >5)):
                        break;
            
            
                if(len_alltables==1 and ('STOCKS AND BONDS' in td.get_text()) | len(td.get_text()) > 50):
                    words = td.get_text().split()
                    index = [i for i, s in enumerate(words) if 'STOCKS' in s]
                    if(len(index)):
                        city = ' '.join(words[0:index[0]])


                if (('bond' in td.get_text().lower()) and countColumn==0): 
                    isBond = True
                    isStock = False
                    isNewBondRow = True
                    if("colspan" in td.attrs):
                        colspan = int(td.attrs['colspan'].encode('utf8'))
                        if(colspan>=2):
                            colSpan = True
                    else:
                        colSpan = False

                    break;

                elif(('stock' in td.get_text().lower() or 'company' in td.get_text().lower()) and countColumn==0):
                    isStock = True
                    isBond = False
                    isNewStockRow = True
                    if("colspan" in td.attrs):
                        colspan = int(td.attrs['colspan'].encode('utf8'))
                        if(colspan>=2):
#                             print(colspan)
                            colSpan = True 
                    else:
                        colSpan = False
                    break;
                if("rowspan" in td.attrs):
                    rowspan = int(td.attrs['rowspan'].encode('utf8'))
                    if(rowspan>=2):
                        rowspan = True
                else:
                    rowspan = False


                txt = td.get_text()
                text = td.get_text().strip()
                text = text.replace('\r','')
                text = text.replace('\n','')
    #             if( (not(colSpan) and countColumn > 0) or (countColumn==1 and txt.strip().isdigit())):
                if(countColumn==1 and txt.strip().isdigit() and isStock):
                    countColumn = countColumn+1
                if(countColumn==1 and (txt.strip().isdigit() or len(txt.strip()) ==0) and isBond and not(colSpan)):
                    countColumn = countColumn+1
                if(countColumn==1 and (txt.strip().isdigit()) and isBond):
                    countColumn = countColumn+1

                if(isBond):
                    if(df_bond_temp.shape[0] == row):
                        if(row!=1 and not pd.isnull(df_bond_temp.loc[row,countColumn])):
                            df_bond_temp.loc[row,countColumn+1] = text
                        else:
                            df_bond_temp.loc[row,countColumn] = text
                    else:
                        df_bond_temp.loc[row,countColumn] = text
                    df_bond_temp.loc[row,'city'] = city 
                elif(isStock):
                    if(df_stock_temp.shape[0] == row):
                        if(row!=1 and not isNewStockRow and not pd.isnull(df_stock_temp.loc[row,countColumn])):
                            df_stock_temp.loc[row,countColumn+1] = text
                        else:
                            df_stock_temp.loc[row,countColumn] = text
                    else:
                        df_stock_temp.loc[row,countColumn] = text
                    df_stock_temp.loc[row,'city'] = city 

                if(rowspan and isBond):
                    df_bond_temp.loc[row+1,countColumn] = text
                if(rowspan and isStock):
                    df_stock_temp.loc[row+1,countColumn] = text




        df_bond = df_bond.append(df_bond_temp)
        df_stock = df_stock.append(df_stock_temp)



    # rename columns
    df_stock.rename(columns={0: 'Company',1: 'Issue', 2: 'Par.',3:'Capital.',4:'Period.',5:'% last div',6:'Date of Issue',7:'Bid.',8:"Ask'd",'city':'City'},inplace=True)
    df_bond.rename(columns={0: 'Company',1: 'Issue', 2: 'Date of Issue',3:'Amount Out–standing',4:'Interest Paid.',5:'%',6:'Principal Due.',7:'Bid.',8:"Ask'd",'city':'City'},inplace=True)


    # add date and page info
    df_bond['Year'] = dateObj['year']
    df_bond['Month'] = dateObj['month']
    df_bond['Day'] = dateObj['day']
    df_bond['Page'] = dateObj['page']

    df_stock['Year'] = dateObj['year']
    df_stock['Month'] = dateObj['month']
    df_stock['Day'] = dateObj['day'] 
    df_stock['Page'] = dateObj['page']



    df_stock=df_stock[['Company','City','Issue', 'Par.','Capital.','Period.','% last div','Date of Issue','Bid.',"Ask'd",'Year','Month','Day','Page']]
    df_bond=df_bond[['Company','City','Issue','Date of Issue','Amount Out–standing','Interest Paid.','%','Principal Due.','Bid.',"Ask'd",'Year','Month','Day','Page']]

    df_stock['% last div'] = df_stock['% last div'].apply(lambda x: convertUnicodeToString2(x))
    df_stock["Ask'd"] = df_stock["Ask'd"].apply(lambda x: convertUnicodeToString2(x))
    df_stock['Bid.'] = df_stock['Bid.'].apply(lambda x: convertUnicodeToString2(x))
    df_stock['Par.'] = df_stock['Par.'].apply(lambda x: convertUnicodeToString2(x))

    df_bond['%'] = df_bond['%'].apply(lambda x: convertUnicodeToString2(x))
    df_bond["Ask'd"] = df_bond["Ask'd"].apply(lambda x: convertUnicodeToString2(x))
    df_bond['Bid.'] = df_bond['Bid.'].apply(lambda x: convertUnicodeToString2(x))

    df_stock['Period.'] = df_stock['Period.'].str.replace(r'(—)', "")
    
    conditions = [
    df_stock['% last div']==99999, df_stock["Ask'd"]==99999 ,df_stock['Bid.']==99999 , df_stock['Par.']==99999 ]
    choices = ['1','1','1','1']
    df_stock['exception']= np.select(conditions, choices, default='')
    
    conditions_bond = [
    df_bond['%']==99999, df_bond["Ask'd"]==99999 ,df_bond['Bid.']==99999]
    choices_bond = ['1','1','1']
    df_bond['exception']= np.select(conditions_bond, choices_bond, default='')
    

    return df_bond, df_stock


            

In [4]:
# this method runs through all the folder and calls fetch data for every html file

from os import listdir
from os.path import join, isfile
# from readFile import readFile

# files=[f for f in listdir('./') if isfile(join('./'),f)]
#  finding all folder names in the current directory
folders=[f for f in listdir('./') if isfile(join('./',f))==False]
# print(folders)

# inspect each folder one by one:
bondsData = pd.DataFrame()
stocksData = pd.DataFrame()
for folder in folders:
    files=[file for file in listdir(folder) if isfile(join(folder,file))]
#     print('i am in ' + folder)
#     print(files)
#     print('-------')

    for file in files:
        dateObj = {}
        dateObj = readFileName(file)
        if('.html' in file):
            bonds,stocks = fetchData(file,folder,dateObj)
#             print(len(bonds))
#             print(len(stocks))
#             if(len(bonds)):
            bondsData = pd.concat([bonds,bondsData],ignore_index=True,sort=False)
#             if(len(stocks)):
            stocksData = pd.concat([stocks,stocksData],ignore_index=True,sort=False)
#             print(len(bondsData))
#             print(len(stocksData))

# stocksData=stocksData[['Company','Issue', 'Par.','Capital.','Period.','% last div','Date of Issue','Bid.',"Ask'd",'Year','Month','Day','Page','Exception']]
# bondsData=bondsData[['Company','Issue','Date of Issue','Amount Out–standing','Interest Paid.','%','Principal Due.','Bid.',"Ask'd",'Year','Month','Day','Page','Exception']]

# this will create the consolidated csv files
bondsData.to_csv('allbonds1_04_20.csv')
stocksData.to_csv('allstocks1_04_20.csv')

In [83]:
# this method is just a test method to run the fetch data logic on one file and see if the output generated is correct or not
# i would change my logic in this method and then would copy the changes to fetch data method.

import pandas as pd
import numpy as np
import codecs
import re
from bs4 import BeautifulSoup

folder = '1892_02_01_vol8(2)'
filename= '1892_02_01_vol8(2)_Page3'


dateObj = readFileName(filename +'.html')
with open('./'+folder+'/'+filename+'.html', 'rb') as f:
    html_doc = f.read()
# html_doc = f.read()
import lxml
soup = BeautifulSoup(html_doc, "lxml")
# t = soup.find('table')

df_bond = pd.DataFrame()
df_stock = pd.DataFrame()

len_alltables = len(soup.find_all('table'))

for table in soup.find_all('table'):
    row = -1
    colSpan = False
    rowspan = False
    nowProceed = False
    isBond = False;
    isStock = False;
    isNewBondRow = False
    isNewStockRow = False
    
    if(len_alltables > 1):
        prev_sibling = table.find_previous_sibling('p')
        print(prev_sibling.text)
        text = prev_sibling.text
        words = text.split()
        index = [i for i, s in enumerate(words) if 'STOCKS' in s or 'SECURITIES' in s]
        if(len(index)):
            city = ' '.join(words[0:index[0]])
        

    
    df_bond_temp = pd.DataFrame()
    df_stock_temp = pd.DataFrame()

    next_td_tag = table.nextSibling
    print(next_td_tag)

    for tr in table.find_all('tr'):
        countColumn = -1
        row = row+1
        countTD = len(tr.find_all('td'))
        for td in tr.find_all('td'):
            countColumn = countColumn + 1
            
#             to handle empty rows
            if(countColumn == 0):
                if(not(len(td.get_text()) >5)):
                    break;

            if(len_alltables==1 and ('STOCKS AND BONDS' in td.get_text()) | len(td.get_text()) > 50):
                words = td.get_text().split()
                index = [i for i, s in enumerate(words) if 'STOCKS' in s]
                if(len(index)):
                    city = ' '.join(words[0:index[0]])
                    print('^^^')
                    print(city)
                
            if (('bond' in td.get_text().lower()) and countColumn==0): 
                print('i m in bond')
                isBond = True
                isStock = False
                isNewBondRow = True
        
                if("colspan" in td.attrs):
                    colspan = int(td.attrs['colspan'].encode('utf8'))
                    if(colspan>=2):
                        colSpan = True
                else:
                    colSpan = False
                
                break;
                
            elif(('stock' in td.get_text().lower() or 'company' in td.get_text().lower()) and countColumn==0):
                print('i m in stock')
                isStock = True
                isBond = False
                isNewStockRow = True           
                if("colspan" in td.attrs):
                    colspan = int(td.attrs['colspan'].encode('utf8'))
                    if(colspan>=2):
                        colSpan = True 
                else:
                    colSpan = False
                break;
            
            
            if("rowspan" in td.attrs):
                rowspan = int(td.attrs['rowspan'].encode('utf8'))
                if(rowspan>=2):
                    rowspan = True
            else:
                rowspan = False
                
            txt = td.get_text()
            text = td.get_text().strip()
            text = text.replace('\r','')
            text = text.replace('\n','')
             
            if(countColumn==1 and txt.strip().isdigit() and isStock):
                countColumn = countColumn+1
            if(countColumn==1 and (txt.strip().isdigit() or len(txt.strip()) ==0) and isBond and not(colSpan)):
                countColumn = countColumn+1
            if(countColumn==1 and (txt.strip().isdigit()) and isBond):
                countColumn = countColumn+1

            if(isBond):
                if(df_bond_temp.shape[0] == row):
                    if(row!=1 and not pd.isnull(df_bond_temp.loc[row,countColumn])):
                        df_bond_temp.loc[row,countColumn+1] = text
                    else:
                        df_bond_temp.loc[row,countColumn] = text
                else:
                    df_bond_temp.loc[row,countColumn] = text
#                 print('HEREEEEE')
#                 print(city)
                df_bond_temp.loc[row,'city'] = city 
            elif(isStock):
                if(df_stock_temp.shape[0] == row):
                    if(row!=1 and not isNewStockRow and not pd.isnull(df_stock_temp.loc[row,countColumn])):
                        df_stock_temp.loc[row,countColumn+1] = text
                    else:
                        df_stock_temp.loc[row,countColumn] = text
                else:
                    df_stock_temp.loc[row,countColumn] = text
                
                df_stock_temp.loc[row,'city'] = city 
            
            if(rowspan and isBond):
                df_bond_temp.loc[row+1,countColumn] = text
            if(rowspan and isStock):
                df_stock_temp.loc[row+1,countColumn] = text
                
            
            
                
            
                
                

    
        print('===============')
    print('***************')
    
    df_bond = df_bond.append(df_bond_temp)
    df_stock = df_stock.append(df_stock_temp)
    
#     print(df_bond)
    
            

# rename columns
df_stock.rename(columns={0: 'Company',1: 'Issue', 2: 'Par.',3:'Capital.',4:'Period.',5:'% last div',6:'Date of Issue',7:'Bid.',8:"Ask'd",'city':'City'},inplace=True)
df_bond.rename(columns={0: 'Company',1: 'Issue', 2: 'Date of Issue',3:'Amount Out–standing',4:'Interest Paid.',5:'%',6:'Principal Due.',7:'Bid.',8:"Ask'd",'city':'City'},inplace=True)

# # df_bond,df_stock = cleanData(df_bond,df_stock)



# add date and page info
df_bond['Year'] = dateObj['year']
df_bond['Month'] = dateObj['month']
df_bond['Day'] = dateObj['day']
df_bond['Page'] = dateObj['page']

df_stock['Year'] = dateObj['year']
df_stock['Month'] = dateObj['month']
df_stock['Day'] = dateObj['day'] 
df_stock['Page'] = dateObj['page']



df_stock=df_stock[['Company','City','Issue', 'Par.','Capital.','Period.','% last div','Date of Issue','Bid.',"Ask'd",'Year','Month','Day','Page']]
df_bond=df_bond[['Company','City','Issue','Date of Issue','Amount Out–standing','Interest Paid.','%','Principal Due.','Bid.',"Ask'd",'Year','Month','Day','Page']]

# df_stock=df_stock[['Company','Issue', 'Par.','Capital.','Period.','% last div','Date of Issue','Bid.',"Ask'd",'Year','Month','Day','Page']]
# df_bond=df_bond[['Company','Issue','Date of Issue','Amount Out–standing','Interest Paid.','%','Principal Due.','Bid.',"Ask'd",'Year','Month','Day','Page']]


df_stock['% last div'] = df_stock['% last div'].apply(lambda x: convertUnicodeToString2(x))
df_stock["Ask'd"] = df_stock["Ask'd"].apply(lambda x: convertUnicodeToString2(x))
df_stock['Bid.'] = df_stock['Bid.'].apply(lambda x: convertUnicodeToString2(x))
df_stock['Par.'] = df_stock['Par.'].apply(lambda x: convertUnicodeToString2(x))

df_bond['%'] = df_bond['%'].apply(lambda x: convertUnicodeToString2(x))
df_bond["Ask'd"] = df_bond["Ask'd"].apply(lambda x: convertUnicodeToString2(x))
df_bond['Bid.'] = df_bond['Bid.'].apply(lambda x: convertUnicodeToString2(x))

df_stock['Period.'] = df_stock['Period.'].str.replace(r'(—)', "")
# df_bond['Ask'] = df_bond['Ask'].str.replace(r'(\*|†|x|-|\:|…|[..]{2,}|%| )', "")

conditions = [
df_stock['% last div']==99999, df_stock["Ask'd"]==99999 ,df_stock['Bid.']==99999 , df_stock['Par.']==99999 ]
choices = ['1','1','1','1']
df_stock['exception']= np.select(conditions, choices, default='')

# df_stock['exception'] = np.select(df_stock['% last div']==99999 or df_stock["Ask'd"]==99999 or df_stock['Bid.']==99999 or df_stock['Par.']==99999 , '1', '')

conditions_bond = [
df_bond['%']==99999, df_bond["Ask'd"]==99999 ,df_bond['Bid.']==99999]
choices_bond = ['1','1','1']
df_bond['exception']= np.select(conditions_bond, choices_bond, default='')



# create csv
df_bond.to_csv('bonds_test_20_04.csv')
df_stock.to_csv('stocks_test_20_04.csv')


print('-----')


OMAHA STOCKS AND BONDS.—Corrected BY
RICHARD C. PATTERSON Banker and Broker, 907 N. Y. Life Building, Omaha, Neb.,
Jan. 18.


i m in stock
***************
 


i m in bond
***************
CINCINNATI STOCKS AND BONDS.—Corrected by GEO.
EUSTIS & Co., Bankers and Brokers, 26 West Third Street, Cincinnati, Jan.
18.


i m in stock
***************
 


i m in bond
***************
BALTIMORE STOCKS AND BONDS.—Corrected by HAMBLETON
& Co, Bankers, 9 South Street, Baltimore. Md.,. Jan. 19.


i m in stock
***************
 


i m in bond
***************
WASHINGTON STOCKS AND BONDS.—Corrected by CRANE,
PARRIS & Co., Bankers, 1344 F Street, N. W. Washington D. C., Jan. 18.


i m in stock
i m in bond
***************
CLEVELAND STOCKS.—Corrected by W. J. HAYES
& SONS, Bankers, Cleveland, O., Jan. 18.


i m in stock
***************
-----
